In [1]:
import tensorflow as tf
import numpy as np

In [6]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [3]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

**1) TF Checkpoint**

Basic logistic regression

In [7]:
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [10]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

        gradients = g.gradient(loss, [W, b])

        optimizer.apply_gradients(zip(gradients, [W, b]))

In [11]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)

    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 497.326050, accuracy: 0.628906
step: 100, loss: 334.016785, accuracy: 0.785156
step: 150, loss: 241.835175, accuracy: 0.832031
step: 200, loss: 157.957886, accuracy: 0.882812
step: 250, loss: 191.231064, accuracy: 0.878906
step: 300, loss: 208.493668, accuracy: 0.847656
step: 350, loss: 184.009323, accuracy: 0.875000
step: 400, loss: 220.533569, accuracy: 0.847656
step: 450, loss: 169.468353, accuracy: 0.878906
step: 500, loss: 117.588249, accuracy: 0.882812
step: 550, loss: 118.535202, accuracy: 0.886719
step: 600, loss: 171.519989, accuracy: 0.851562
step: 650, loss: 141.058090, accuracy: 0.878906
step: 700, loss: 87.067368, accuracy: 0.898438
step: 750, loss: 177.132095, accuracy: 0.855469
step: 800, loss: 52.242306, accuracy: 0.929688
step: 850, loss: 106.422684, accuracy: 0.886719
step: 900, loss: 176.263107, accuracy: 0.863281
step: 950, loss: 69.676392, accuracy: 0.921875
step: 1000, loss: 99.548164, accuracy: 0.910156


Save and Load with TF Checkpoint

In [17]:
vars_to_save = {"W": W, "b": b, "optimizer": optimizer}
checkpoint = tf.train.Checkpoint(**vars_to_save)
saver = tf.train.CheckpointManager(
    checkpoint, directory="./tf-example", max_to_keep=5
)

In [18]:
saver.save()

'./tf-example/ckpt-1'

In [19]:
np.mean(W.numpy())

-0.09217432

In [20]:
# Reset variables to test restore.
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [21]:
# Check resetted weight value.
np.mean(W.numpy())

-0.0158106

In [23]:
vars_to_load = {"W": W, "b": b, "optimizer": optimizer}
checkpoint = tf.train.Checkpoint(**vars_to_load)
latest_ckpt = tf.train.latest_checkpoint("./tf-example")
checkpoint.restore(latest_ckpt)

In [24]:
np.mean(W.numpy())

-0.09217432

**2) TF Model**

Basic neural network with TF Model

In [25]:
from tensorflow.keras import Model, layers

In [26]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 100

In [27]:
# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__(name="NeuralNet")
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(64, activation=tf.nn.relu)
        # Second fully-connected hidden layer.
        self.fc2 = layers.Dense(128, activation=tf.nn.relu)
        # Third fully-connecter hidden layer.
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass.
    def __call__(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [28]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(crossentropy)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [29]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy(pred, y)

        # Compute gradients.
        gradients = g.gradient(loss, neural_net.trainable_variables)

        # Update W and b following gradients.
        optimizer.apply_gradients(zip(gradients, neural_net.trainable_variables))

In [30]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=False)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.192535, accuracy: 0.859375
step: 200, loss: 2.193727, accuracy: 0.859375
step: 300, loss: 2.182334, accuracy: 0.933594
step: 400, loss: 2.179169, accuracy: 0.953125
step: 500, loss: 2.176396, accuracy: 0.972656
step: 600, loss: 2.177567, accuracy: 0.964844
step: 700, loss: 2.175521, accuracy: 0.976562
step: 800, loss: 2.177242, accuracy: 0.964844
step: 900, loss: 2.176783, accuracy: 0.972656
step: 1000, loss: 2.177277, accuracy: 0.964844


Save and Load with TF Model

In [31]:
neural_net.save_weights(filepath="./tfmodel.ckpt")

In [33]:
neural_net = NeuralNet()
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.062500


In [34]:
# Load saved weights.
neural_net.load_weights(filepath="./tfmodel.ckpt")

In [35]:
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))

accuracy: 0.964844
